In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul 26 13:06:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import pandas_datareader as pdr
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense ,LSTM, Dropout,Bidirectional,TimeDistributed
from sklearn.model_selection import train_test_split, KFold
from tensorflow import expand_dims
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from os.path import join
from tensorflow import constant
from time import sleep
from prophet import Prophet
#import FinanceDataReader as fdr

2023-07-27 01:16:50.348690: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-27 01:16:50.421336: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 01:16:50.749924: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 01:16:50.751488: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 01:16:51.813985: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
data=pd.read_csv('/home/junseo/Desktop/2023_1st_vacation/KRX_py/KRX_DATA/open/train.csv')

In [4]:
gl=pd.read_csv('/home/junseo/Desktop/2023_1st_vacation/KRX_py/KRX_DATA/open/goldlee.csv')

In [5]:
def convert_date(date_str):
  return pd.to_datetime(date_str, format="%Y%m%d").strftime("%Y-%m-%d")
data["일자"] =data["일자"].apply(convert_date)

In [6]:
def convert_date(date_str):
  return pd.to_datetime(date_str, format="%Y/%m/%d").strftime("%Y-%m-%d")
gl["일자"] =gl["일자"].apply(convert_date)

In [7]:
gl = gl[gl["일자"].isin(data["일자"].unique())]
gl.set_index('일자',inplace=True)

In [8]:
# prescaler
def getAOC(data):
  AOC = []
  close = data['종가']
  close_pre = close.shift(1)

  AOC.append(0)

  # aoc = (금일 종가 - 전일 종가) / 전일 종가
  for i in range(1, len(close)):
    aoc_value = ((close[i] - close_pre[i]) / close_pre[i]) * 100
    AOC.append(aoc_value)

  return AOC

#sort data by "종목명"
subject_Data=data.sort_values(by="종목코드",inplace=False)
subject_Data.head()
subject_Data.columns


pv_s_data=pd.pivot_table(subject_Data,values=['거래량', '시가', '고가', '저가', '종가'],index=['종목코드','일자'])
pv_s_data["AOC"] = getAOC(pv_s_data)

divided = pv_s_data.index.get_level_values('종목코드').unique()

dataframes = []

for i in divided:
    sliced_data = pv_s_data.loc[i]
    sliced_dataframe = sliced_data.reset_index()
    dataframes.append(sliced_dataframe)

sliced_dataframes=[]

for s in dataframes:
    end = s.pop('종가')
    sliced_dataframe = pd.merge(s, gl, left_on='일자', right_index=True, how='left')
    sliced_dataframe['종가'] = end
    sliced_dataframes.append(sliced_dataframe)

In [9]:
data

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,2021-06-01,A060310,3S,166690,2890,2970,2885,2920
1,2021-06-01,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,2021-06-01,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,2021-06-01,A054620,APS,462544,14600,14950,13800,14950
4,2021-06-01,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,2023-05-30,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,2023-05-30,A000540,흥국화재,50218,3250,3255,3195,3215
987997,2023-05-30,A003280,흥아해운,130664,1344,1395,1340,1370
987998,2023-05-30,A037440,희림,141932,9170,9260,9170,9200


In [10]:
test=sliced_dataframes[1999]
test

,일자,거래량,고가,시가,저가,AOC,한국은행 기준금리,정부대출금금리,무역금융지원 프로그램대출금리,영세자영업자지원 프로그램대출금리,신성장·일자리지원 프로그램대출금리,설비투자지원 프로그램대출금리,지방중소기업지원 프로그램대출금리,자금조정 대출금리,자금조정 예금금리,종가
0,2021-06-01,1879288,11300,11000,10900,-82.258065,0.5,0.601,0.25,0.25,0.25,0.25,0.25,1.0,0.0,11000
1,2021-06-02,2426922,10850,10700,10600,-2.727273,0.5,0.601,0.25,0.25,0.25,0.25,0.25,1.0,0.0,10700
2,2021-06-03,2709800,10700,10650,10300,-2.803738,0.5,0.601,0.25,0.25,0.25,0.25,0.25,1.0,0.0,10400
3,2021-06-04,1737593,10650,10450,10350,0.480769,0.5,0.601,0.25,0.25,0.25,0.25,0.25,1.0,0.0,10450
4,2021-06-07,2714980,11150,10550,10500,3.349282,0.5,0.601,0.25,0.25,0.25,0.25,0.25,1.0,0.0,10800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,2023-05-23,150364,8390,8390,8310,-0.358852,3.5,3.534,2.00,2.00,2.00,2.00,2.00,4.5,2.5,8330
490,2023-05-24,122457,8340,8310,8280,-0.360144,3.5,3.534,2.00,2.00,2.00,2.00,2.00,4.5,2.5,8300
491,2023-05-25,84241,8310,8300,8270,0.120482,3.5,3.534,2.00,2.00,2.00,2.00,2.00,4.5,2.5,8310
492,2023-05-26,126681,8310,8300,8270,-0.361011,3.5,3.534,2.00,2.00,2.00,2.00,2.00,4.5,2.5,8280


In [11]:
## sorting labels
labels=data[['종목코드','일자']]
labels = labels.sort_values(by=['종목코드','일자'])
labels=labels.drop('일자',axis=1)
labels=labels.drop_duplicates('종목코드')
data.sort_values(by=['종목코드',"일자"],inplace=True)

In [12]:
label_lis=[]
for label in labels:
  label_lis.append(label)

In [13]:
for sliced in sliced_dataframes:
  sliced.drop('일자',axis=1,inplace=True)

In [14]:
sliced_dataframes[0]

,거래량,고가,시가,저가,AOC,한국은행 기준금리,정부대출금금리,무역금융지원 프로그램대출금리,영세자영업자지원 프로그램대출금리,신성장·일자리지원 프로그램대출금리,설비투자지원 프로그램대출금리,지방중소기업지원 프로그램대출금리,자금조정 대출금리,자금조정 예금금리,종가
0,114966,14700,14700,14450,0.000000,0.5,0.601,0.25,0.25,0.25,0.25,0.25,1.0,0.0,14600
1,109559,14700,14700,14450,-0.684932,0.5,0.601,0.25,0.25,0.25,0.25,0.25,1.0,0.0,14500
2,96158,14650,14550,14450,0.689655,0.5,0.601,0.25,0.25,0.25,0.25,0.25,1.0,0.0,14600
3,133900,14800,14600,14550,0.684932,0.5,0.601,0.25,0.25,0.25,0.25,0.25,1.0,0.0,14700
4,511140,15550,14800,14750,3.061224,0.5,0.601,0.25,0.25,0.25,0.25,0.25,1.0,0.0,15150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,641524,9900,9160,9160,5.280172,3.5,3.534,2.00,2.00,2.00,2.00,2.00,4.5,2.5,9770
490,205243,9820,9770,9550,-0.307062,3.5,3.534,2.00,2.00,2.00,2.00,2.00,4.5,2.5,9740
491,398326,10180,9660,9660,3.080082,3.5,3.534,2.00,2.00,2.00,2.00,2.00,4.5,2.5,10040
492,196257,10150,10050,9850,-1.892430,3.5,3.534,2.00,2.00,2.00,2.00,2.00,4.5,2.5,9850


## Model shells

In [18]:
class LSTM2():
  def __init__(self,data,idx):
    self.data=data
    self.idx=idx
    self.time_steps=60

  def create_sequences(self, data):
    num_samples, num_features = data.shape
    sequences = []
    for i in range(num_samples - self.time_steps +1):
      sequences.append(data[i:i + self.time_steps, :])
    return np.array(sequences)

  def inverse_sequences(self,data_sequences):
    num_samples, num_steps, num_features = data_sequences.shape
    data = np.zeros((num_samples + self.time_steps - 1, num_features))
    for i in range(num_samples):
      data[i:i + self.time_steps, :] += data_sequences[i, :, :]
    data /= self.time_steps
    return data[:num_samples]
    #frquency domain inverse laplace transform

  def slicing_data(self):
    self.x_scaler = MinMaxScaler()
    self.y_scaler = MinMaxScaler()
    # Scaling features
    x_data = self.data[self.data.columns[:]]
    x_data = self.x_scaler.fit_transform(x_data)
    # Scaling target variable
    y_data = self.data[self.data.columns[:]]
    y_data = self.y_scaler.fit_transform(y_data)
    # validation set
    x_train_p, x_test, y_train_p, y_test = train_test_split(x_data, y_data, test_size=0.2, shuffle=False)
    x_train, x_val, y_train, y_val = train_test_split(x_train_p, y_train_p, test_size=0.25, shuffle=False)
    __t,x_final,__t,__t=train_test_split(x_data, y_data, test_size=60/len(y_data), shuffle=False)

    self.num_features = x_data.shape[1]
    self.num_samples=x_data.shape[0]

    self.x_train = self.create_sequences(x_train)
    self.x_test = self.create_sequences(x_test)
    self.x_val = self.create_sequences(x_val)
    self.x_final = self.create_sequences(x_final)

    self.y_train=self.create_sequences(y_train)
    self.y_test=self.create_sequences(y_test)
    self.y_val=self.create_sequences(y_val)

  def model_struct(self):
    self.model = Sequential()
    # layers
    self.model.add(Bidirectional(LSTM(1024, return_sequences=True, input_shape=(self.time_steps, self.num_features))))
    self.model.add(Dense(256, activation='relu'))
    self.model.add(Dense(128, activation='relu'))
    self.model.add(Dense(64, activation='relu'))
    self.model.add(Dense(32, activation='relu'))
    self.model.add(Dense(16, activation='relu'))
    # output layer (many-to-many with TimeDistributed)
    self.model.add(TimeDistributed(Dense(len(self.data.columns), activation='relu')))



  def learning(self):
    model_save_path = '/home/junseo/Desktop/2023_1st_vacation/KRX_py/ckptrs/'

    self.filename = join(model_save_path, 'ckpointer_0727_{}.ckpt'.format(self.idx))
    checkpoint = ModelCheckpoint(self.filename, save_weights_only=True, save_best_only=True, monitor='val_loss', verbose=0)
    earlystopping = EarlyStopping(monitor='val_loss', patience=100)
    self.model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
    self.history = self.model.fit(self.x_train, self.y_train, epochs=1, batch_size=128, validation_data=(self.x_val, self.y_val), shuffle=False, callbacks=[checkpoint, earlystopping], verbose=0)

  def get_gap_by_test(self):
    self.model.load_weights(self.filename)
    pred = self.model.predict(self.x_test,verbose=0)
    rescaled_pred = self.y_scaler.inverse_transform(self.inverse_sequences(pred))
    rescaled_real = self.y_scaler.inverse_transform(self.inverse_sequences(self.y_test))

    gaps = []

    for p, r in zip(rescaled_pred, rescaled_real):
        gap = np.abs(p - r)
        gaps.append(gap)

    avg_gap = np.mean(gaps)

    return avg_gap



  def return_val(self):
      self.slicing_data()
      self.model_struct()
      self.learning()
      self.model.load_weights(self.filename)

      next_input_data = np.copy(self.x_final)
      for day in range(1, 16):
          pred_day = self.model.predict(np.expand_dims(next_input_data[-1], axis=0),verbose=0)
          next_input_data = np.concatenate((next_input_data, pred_day), axis=0)

      next_15_days_data = next_input_data[-15:]
      next_15_days_data=self.y_scaler.inverse_transform(self.inverse_sequences(next_15_days_data))

      columns=self.data.columns.tolist()
      ret_df = pd.DataFrame(next_15_days_data, columns=columns)

      prices=[]
      for end_price in ret_df.iloc[:,-1]:
        prices.append(end_price)

      gap=self.get_gap_by_test()


      return prices,gap

## TEST **Shell**

In [21]:
l1=[]
l2=[]
#test shell
m=LSTM2(sliced_dataframes[0],0)

m.slicing_data()

m.model_struct()

m.learning()
pred,gap1=m.return_val()
l1.append(pred)
l2.append(gap1)
m=LSTM2(sliced_dataframes[1],1)
pred1,gap=m.return_val()
l1.append(pred1)
l2.append(gap)

df=pd.DataFrame({
    '종가':l1,
    'gap':l2
})
df.to_csv('/home/junseo/Desktop/2023_1st_vacation/KRX_py/test/{}.csv'.format("230726_v0.1"),index=True)

2023-07-27 01:22:16.266254: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-27 01:22:16.267777: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-27 01:22:16.268825: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-07-27 01:22:24.110708: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-27 01:22:24.112444: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-27 01:22:24.114075: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-07-27 01:22:27.306119: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-27 01:22:27.307372: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-27 01:22:27.308212: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

## Predict **Shell**

In [22]:
idx=0

pred_list=[]
gap_acc=[]
out_labels=[]

In [23]:
for c in sliced_dataframes:
  print(labels.iloc[idx])
  print("{}번째 종목코드".format(idx))
  LSTM_model=LSTM2(c,idx=idx)
  pred,gap = LSTM_model.return_val()
  sleep(1)
  pred_list.append(pred)
  gap_acc.append(gap)
  idx+=1
  #to prevent model corrupting

종목코드    A000020
Name: 502, dtype: object
0번째 종목코드


2023-07-27 01:23:18.680170: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-27 01:23:18.681741: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-27 01:23:18.682755: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

종목코드    A000040
Name: 118, dtype: object
1번째 종목코드


2023-07-27 01:23:33.832026: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-27 01:23:33.833512: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-27 01:23:33.834427: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

종목코드    A000050
Name: 243, dtype: object
2번째 종목코드


...

2023-07-27 01:50:33.244861: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-27 01:50:33.245945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-27 01:50:33.247010: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

종목코드    A002310
Name: 988, dtype: object
104번째 종목코드


2023-07-27 01:50:49.342415: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-27 01:50:49.343781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-27 01:50:49.345138: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

종목코드    A002320
Name: 1893, dtype: object
105번째 종목코드


2023-07-27 01:51:05.076761: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-27 01:51:05.078135: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-27 01:51:05.079113: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
pred_csv_df=pd.DataFrame({
    '종목코드':label_lis,
    '종가':pred_list,
    'gap':gap_acc
})

In [ ]:
pred_csv_df.to_csv('/home/junseo/Desktop/2023_1st_vacation/KRX_py/csvs/{}.csv'.format("230725_v0.1"),index=True)

In [ ]:
import os
os.system('shutdown -s -f')
